In [13]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
import matplotlib.pyplot as plt

#from data import get_data

In [14]:
def get_data(file, nrows=None):
    if file == 'listings':
        cols = [ 'id',
                 'name',
                 'summary',
                 'space',
                 'description',
                 'experiences_offered',
                 'neighborhood_overview',
                 'notes',
                 'transit',
                 'access',
                 'interaction',
                 'house_rules',
                 'host_since',
                 'host_location',
                 'host_about',
                 'host_response_time',
                 'host_response_rate',
                 'host_neighbourhood',
                 'host_listings_count',
                 'host_total_listings_count',
                 'host_verifications',
                 'host_identity_verified',
                 'street',
                 'neighbourhood',
                 'neighbourhood_cleansed',
                 'neighbourhood_group_cleansed',
                 'zipcode',
                 'latitude',
                 'longitude',
                 'is_location_exact',
                 'property_type',
                 'room_type',
                 'accommodates',
                 'bathrooms',
                 'bedrooms',
                 'beds',
                 'bed_type',
                 'amenities',
                 'price',
                 'weekly_price',
                 'monthly_price',
                 'security_deposit',
                 'cleaning_fee',
                 'guests_included',
                 'extra_people',
                 'minimum_nights',
                 'maximum_nights',
                 'availability_30',
                 'availability_60',
                 'availability_90',
                 'availability_365',
                 'number_of_reviews',
                 'number_of_reviews_ltm',
                 'first_review',
                 'last_review',
                 'review_scores_rating',
                 'review_scores_accuracy',
                 'review_scores_cleanliness',
                 'review_scores_checkin',
                 'review_scores_communication',
                 'review_scores_location',
                 'review_scores_value',
                 'instant_bookable',
                 'cancellation_policy',
                 'require_guest_profile_picture',
                 'require_guest_phone_verification',
                 'reviews_per_month']
        dates = ['host_since', 'first_review', 'last_review']
        data = pd.read_csv('listings_january.csv', low_memory=False, parse_dates=dates, usecols=cols,
                           index_col='id', nrows=nrows)
    return data

listings = get_data('listings')

In [21]:
def decode_amenities(df):
    data = df
    amenities = data['amenities']
    amenity_rows = []
    for row in amenities:
        row_items = row[1:-1].split(',')
        for key,item in enumerate(row_items):
            if item[0] == '"':
                row_items[key] = item[1:-1]
        amenity_rows.append(row_items)
    return amenity_rows

In [15]:
listings.head()

,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,host_since,host_location,host_about,host_response_time,host_response_rate,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11551,Arty and Bright London Apartment in Zone 2,Unlike most rental apartments out there my fla...,"Amenities Bedding: 1 Double bed, 1 living room...",Unlike most rental apartments out there my fla...,family,Not even 10 minutes by metro from Victoria Sta...,No Smoking (very strict) Check-in time is afte...,Tons of buses (24hrs) go into central London f...,Guest will have access to the entire apartment,No interaction with guests as you book the ent...,No Smoking (very strict) No pets are allowed i...,2009-10-03,"London, England, United Kingdom","Hello, I'm a friendly Italian man with a very ...",within an hour,100%,Brixton,3.0,3.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,"London, United Kingdom",Brixton,Lambeth,NaN,SW9 8DG,51.46225,-0.11732,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Paid par...",$88.00,$645.00,"$2,350.00",$400.00,$30.00,2,$25.00,2,180,17,42,72,344,186,3,2010-03-21,2019-11-17,92.0,9.0,9.0,10.0,10.0,9.0,9.0,t,strict_14_with_grace_period,f,t,1.56
13913,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,"Hello Everyone, I'm offering my lovely double ...",My bright double bedroom with a large window h...,business,Finsbury Park is a friendly melting pot commun...,For art lovers I can give guest my Tate Member...,The flat only a 10 minute walk to Finsbury Par...,Guest will have access to the self catering ki...,I like to have little chats with my guest over...,I'm an artist and have my artwork up on the wa...,2009-11-16,"London, England, United Kingdom",I am a Multi-Media Visual Artist and Creative ...,within a few hours,70%,LB of Islington,4.0,4.0,"['email', 'phone', 'facebook', 'reviews', 'off...",t,"Islington, Greater London, United Kingdom",LB of Islington,Islington,NaN,N4 3,51.56802,-0.11121,t,Apartment,Private room,2,1.0,1.0,0.0,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,""Paid parking off ...",$65.00,$333.00,"$1,176.00",$100.00,$15.00,1,$15.00,1,29,30,60,90,365,20,6,2010-08-18,2019-11-25,97.0,10.0,10.0,10.0,10.0,9.0,9.0,f,moderate,f,f,0.17
15400,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,Bright Chelsea Apartment This is a bright one...,Lots of windows and light. St Luke's Gardens ...,romantic,It is Chelsea.,The building next door is in the process of be...,The underground stations are South Kensington ...,There are two wardrobes for guests exclusive u...,If I am in the country I like to welcome my gu...,NO SMOKING PLEASE.. No unauthorised guests. No...,2009-12-05,"Kensington, England, United Kingdom","English, grandmother, I have travelled quite ...",within a few hours,100%,Chelsea,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,"London, United Kingdom",RB of Kensington and Chelsea,Kensington and Chelsea,NaN,SW3,51.48796,-0.16898,t,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,

In [28]:
listings.shape

(87235, 66)

In [23]:
def decode_amenities(df):
    data = df   
    def str_to_array(strn):
        row_items = strn[1:-1].split(',')
        for key,item in enumerate(row_items):
            row_items[key] = item.strip('"').casefold()
        return row_items    
    return data[['amenities']].applymap(str_to_array)
def count_amenities(df):
    data = df
    amenities = decode_amenities(data)
    return amenities.applymap(lambda x: len(x))
def has_amenity(df, name, alias=None):
    data = df
    if not alias:
        alias = name
    amenities = decode_amenities(data)
    col_name = f'has_{alias}'
    amenities[col_name] = amenities[['amenities']].applymap(lambda x: 1 if name.casefold() in x else 0)
    return amenities[[col_name]]
def count_amenity(df, name):
    data = has_amenity(df, name)
    return int(data.sum())

In [26]:
count_amenity(listings, 'heating')

82490

In [30]:
am=decode_amenities(listings)

In [36]:
def get_amenities_list(data):

    av_amenities=[]

    for row in decode_amenities(data)['amenities']:
        for item in row:
            if item not in av_amenities:
                av_amenities.append(item)
    return av_amenities
  

In [37]:
av=get_amenities_list(listings)

In [38]:
av

['tv',
 'cable tv',
 'internet',
 'wifi',
 'kitchen',
 'paid parking off premises',
 'buzzer/wireless intercom',
 'heating',
 'family/kid friendly',
 'washer',
 'smoke detector',
 'carbon monoxide detector',
 'essentials',
 'hangers',
 'hair dryer',
 'iron',
 'laptop friendly workspace',
 'self check-in',
 'lockbox',
 'pack ’n play/travel crib',
 'hot water',
 'long term stays allowed',
 'smoking allowed',
 'free street parking',
 'dryer',
 'fire extinguisher',
 'shampoo',
 'lock on bedroom door',
 'outlet covers',
 'bathtub',
 'children’s books and toys',
 'babysitter recommendations',
 'crib',
 'room-darkening shades',
 'children’s dinnerware',
 'bed linens',
 'extra pillows and blankets',
 'ethernet connection',
 'coffee maker',
 'refrigerator',
 'dishes and silverware',
 'cooking basics',
 'oven',
 'stove',
 'patio or balcony',
 'luggage dropoff allowed',
 'host greets you',
 'handheld shower head',
 'air conditioning',
 '24-hour check-in',
 'translation missing: en.hosting_amenity

In [1]:
filterable_features = ['Free parking on premises', 'Hot tub', 'Gym', 'Pool']
filterable_amenities = ['Kitchen','Shampoo','Heating','Air conditioning','Washing machine','Dryer','Wifi','Breakfast',
                        'Indoor fireplace','Hangers','Iron','Hair dryer','Dedicated workspace','TV','Cot','High chair',
                        'Self check-in','Smoke alarm','Carbon monoxide alarm','Private bathroom','Piano']
localized = [['Washer','Washer / Dryer'], ["Pack 'n Play/travel crib', 'crib"], 'Smoke detector', 'carbon monoxide detector',
            'laptop friendly workspace']

SyntaxError: invalid syntax (<ipython-input-1-866a3e8955b4>, line 5)

In [ ]:
['Free parking on premises']

In [ ]:
['Hot tub','shared hot tub','private hot tub']

In [ ]:
['Gym','shared gym']

In [ ]:
['Pool','private pool','shared pool']

In [ ]:
['Kitchen','kitchenette','full kitchen',"chef's kitchen"]

In [ ]:
['tv', 'cable tv']

In [ ]:
filterable_features_localized = ['Free parking on premises', ['Hot tub','shared hot tub','private hot tub'],
                      ['Gym','shared gym'], ['Pool','private pool','shared pool']]
filterable_amenities_localized = [['Kitchen','kitchenette','full kitchen',"chef's kitchen"],'Shampoo','Heating','Air conditioning',
                        ['Washing machine','Washer','Washer / Dryer'],'Dryer','Wifi','Breakfast',
                        'Indoor fireplace','Hangers','Iron','Hair dryer',['Dedicated workspace','laptop friendly workspace'],
                        ['tv', 'cable tv'],['Cot',"Pack 'n Play/travel crib', 'crib"],'High chair',
                        'Self check-in',['Smoke alarm','Smoke detector'],['Carbon monoxide alarm','carbon monoxide detector'],
                        'Private bathroom','Piano']